In [1]:
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import imageio
from skimage.transform import rescale
from skimage import exposure
from torch.utils import data
import pickle
from torchvision import transforms
from kymatio import Scattering2D
import torch
from sklearn.svm import SVC,LinearSVC
os.environ["KYMATIO_BACKEND"] = "skcuda"
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

In [2]:
print(torch.__version__)


0.4.0


In [3]:
class ep(data.Dataset):
    def __init__(self, parent_dir, lesion = 'MA',transform=None,train = True,load_prob = True,green = False ,rescale = 'intensity'):
        """
        Args:
            parent_dir: directory up to /e_optha/ ,e.x. /Users/jialing/Downloads/1013pj/data/e_optha/
            lesion: 'MA' or 'EX', 'MA' has more samples
            transform (optional): Optional transform to be applied on a sample.
            green: If true, only take the green channel
            resclae: 'intensenty' or 'equilization'
        """
        self.green = green
        self.rescale = rescale
        self.transform = transform
        
        if not parent_dir.endswith('/'):
            # Make sure the directory name is correctly given
            parent_dir = parent_dir + '/'
        parent_dir += ('e_optha_' + lesion)
 
        data_list = []
        for lab in ['/healthy/','/MA/']:
            filelist  = glob.glob(parent_dir +lab+ '**/*.jpg', recursive=True)
            filelist.extend(glob.glob(parent_dir +lab+'**/*.JPG', recursive=True))
            data_list.extend([(file,0) if lab == '/healthy/' else (file,1) for file in filelist]) #include labels before random split
        
        self.data_list = data_list #All the (filename,label)
        
#         self.num_allsamples = len(data_list)
#         self.transform = transform

        random.seed(3)
        random.shuffle(data_list)       
        trainlst = data_list[:round(0.8*len(data_list))]
        testlst = data_list[round(0.8*len(data_list)):]
        
        if train:
            jpg_list = trainlst
        else:
            jpg_list = testlst
        
        self.image_data_dict = {}

        phase = 'patchtrain' if train else 'patchtest'
        if load_prob:
            f_myfile = open(phase + '.pickle', 'rb')
            self.image_data_dict = pickle.load(f_myfile)
            f_myfile.close()
        else:
            for i in range(len(jpg_list)):
                dataset = imageio.imread(jpg_list[i][0])
                self.image_data_dict[i] = [dataset, jpg_list[i][1]] #pixle and label
            with open(phase + '.pickle', 'wb') as handle:
                pickle.dump(self.image_data_dict, handle)

    def __len__(self):
        return len(self.image_data_dict)

    def __getitem__(self,index):
        '''
        Return a tuple containing the image tensor and corresponding class for the given index.
        Parameter:
        index: This is the index created by _init_, it's the key of the dict in _init_
               Notice that a single patient could have multiple index associated.
        '''
        if index not in self.image_data_dict:
            raise ValueError('Index out of bound')
        img,tag = self.image_data_dict[index]
        
        
        #rescale, this will keep the np.uint8 type
        if self.rescale == 'intensity':
            p2, p98 = np.percentile(img, (2, 98))
            img = exposure.rescale_intensity(img, in_range=(p2, p98))
        elif self.rescale == 'equilization':
            img = exposure.equalize_hist(img)
       
      
       #isolating green channel:
        if self.green:
            img = img[:,:,1]
            
        img = transforms.ToPILImage()(img)
        img = transforms.functional.resize(img,160)
        
        if self.transform:
            img = self.transform(img)
        
        sample = (img, tag)
        
        return sample

        
# def main():
#     imginfo = ep('/scratch/jx1047/project/Scattering-colonography/data/e_optha/'
#        ,transform = transforms.Compose([transforms.CenterCrop(180),transforms.ToTensor()
#                                         ,transforms.Normalize((0.1616, ),(0.14998825,))])
#        ,train = False,load_prob = False,green = True
#        ,rescale = 'intensity')
#     use_cuda = torch.cuda.is_available()
#     device = torch.cuda.device("cuda:0" if use_cuda else "cpu")
    

#     params = {'batch_size': 1, 
#           'shuffle': True,
#           'num_workers': 4}
#     training_generator = data.DataLoader(imginfo, **params)
#     #scattering = Scattering2D(J=2, shape=(180,180)) 
#     #image,label = iter(training_generator).next()
#     #exp = scattering(image) #batch scattering
#     return training_generator

    
# if __name__ == "__main__":
#     loader = main()


#transforms.Normalize((0.3080, 0.1616, 0.0848),(0.26722345,0.14998825,0.08663067))




In [83]:

# random.shuffle(t.data_list)

# negcount = 0
# poscount = 0
# for i in t.image_data_dict.values():
#     if i[1] == 0:
#         negcount += 1
#     else:
#         poscount +=1
# print(negcount,poscount,poscount/(poscount+negcount)) #approximate the global base rate

torch.Size([1, 180, 180])

# Scattering on SVM, MA dataset, three channel.

In [91]:
scattering = Scattering2D(J=2, shape=(180,180)) 
for i, a in enumerate(loader, 0):
    ex = scattering(a[0])
    if i == 0:
        xtrain = ex.numpy().flatten()
        ytrain = a[1].numpy()
    else:
        xtrain = np.vstack((xtrain,ex.numpy().flatten()))
        ytrain = np.vstack((ytrain,a[1].numpy()))
        
ytrain = ytrain.reshape(-1)    
 #get scattering representation of training images: flatten all the coefficients across three channels
#get representation both in three channles and in green channel.

In [93]:
print(xtrain.shape,ytrain.shape)

(305, 164025) (305,)


In [94]:
with open('scattrainx_green.pickle','wb') as handle:
    pickle.dump(xtrain,handle)
    
with open('scattrainy_green.pickle','wb') as handle:
    pickle.dump(ytrain,handle)  

In [97]:
for i, a in enumerate(loader, 0):
    ex = scattering(a[0])
    if i == 0:
        xtest = ex.numpy().flatten()
        ytest = a[1].numpy()
    else:
        xtest = np.vstack((xtest,ex.numpy().flatten()))
        ytest = np.vstack((ytest,a[1].numpy()))
ytest = ytest.reshape(-1)

print(xtest.shape,ytest.shape)

(76, 164025) (76,)


In [98]:
with open('scattestx_green.pickle','wb') as handle:
    pickle.dump(xtest,handle)
    
with open('scattesty_green.pickle','wb') as handle:
    pickle.dump(ytest,handle)  

# GridsearchCV for SVM PCA

In [184]:
file = open('scattrainx.pickle','rb')
xtrain = pickle.load(file)
file = open('scattrainy.pickle','rb')
ytrain = pickle.load(file).reshape(-1)
file = open('scattestx.pickle','rb')
xtest = pickle.load(file)
file = open('scattesty.pickle','rb')
ytest = pickle.load(file).reshape(-1)

scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
scaler = StandardScaler()
scaler.fit(xtrain)
scaler.transform(xtrain)
scaler.transform(xtest)
# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score.
# That estimator is made available at ``gs.best_estimator_`` along with
# parameters like ``gs.best_score_``, ``gs.best_params_`` and
# ``gs.best_index_``
pipe = Pipeline([
    # the reduce_dim stage is populated by the param_grid
    ('reduce_dim', PCA()),
    ('classify', SVC())
])

param_grid = dict(reduce_dim__n_components=[1500,1600,1700,1800],classify__C = [2.4,2.5,2.6,2.8])


grid = GridSearchCV(pipe, cv=5, n_jobs=1, 
                    param_grid=param_grid,scoring=scoring, refit='AUC', return_train_score=True)

grid.fit(xtrain, ytrain)
results = grid.cv_results_

In [186]:
results.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_classify__C', 'param_reduce_dim__n_components', 'params', 'split0_test_AUC', 'split1_test_AUC', 'split2_test_AUC', 'split3_test_AUC', 'split4_test_AUC', 'mean_test_AUC', 'std_test_AUC', 'rank_test_AUC', 'split0_train_AUC', 'split1_train_AUC', 'split2_train_AUC', 'split3_train_AUC', 'split4_train_AUC', 'mean_train_AUC', 'std_train_AUC', 'split0_test_Accuracy', 'split1_test_Accuracy', 'split2_test_Accuracy', 'split3_test_Accuracy', 'split4_test_Accuracy', 'mean_test_Accuracy', 'std_test_Accuracy', 'rank_test_Accuracy', 'split0_train_Accuracy', 'split1_train_Accuracy', 'split2_train_Accuracy', 'split3_train_Accuracy', 'split4_train_Accuracy', 'mean_train_Accuracy', 'std_train_Accuracy'])

In [187]:
grid.best_estimator_

Pipeline(memory=None,
     steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=1500, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classify', SVC(C=2.4, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [175]:
results['mean_test_AUC']

array([ 0.71775673,  0.71775673,  0.71775673,  0.71775673,  0.718661  ,
        0.718661  ,  0.718661  ,  0.718661  ,  0.71842126,  0.71842126,
        0.71842126,  0.71842126,  0.71749739,  0.71749739,  0.71749739,
        0.71749739])

In [176]:
results['params']

[{'classify__C': 2.4, 'reduce_dim__n_components': 1500},
 {'classify__C': 2.4, 'reduce_dim__n_components': 1600},
 {'classify__C': 2.4, 'reduce_dim__n_components': 1700},
 {'classify__C': 2.4, 'reduce_dim__n_components': 1800},
 {'classify__C': 2.5, 'reduce_dim__n_components': 1500},
 {'classify__C': 2.5, 'reduce_dim__n_components': 1600},
 {'classify__C': 2.5, 'reduce_dim__n_components': 1700},
 {'classify__C': 2.5, 'reduce_dim__n_components': 1800},
 {'classify__C': 2.6, 'reduce_dim__n_components': 1500},
 {'classify__C': 2.6, 'reduce_dim__n_components': 1600},
 {'classify__C': 2.6, 'reduce_dim__n_components': 1700},
 {'classify__C': 2.6, 'reduce_dim__n_components': 1800},
 {'classify__C': 2.8, 'reduce_dim__n_components': 1500},
 {'classify__C': 2.8, 'reduce_dim__n_components': 1600},
 {'classify__C': 2.8, 'reduce_dim__n_components': 1700},
 {'classify__C': 2.8, 'reduce_dim__n_components': 1800}]

# Patches

In [4]:
file = open('patchtrain.pickle','rb')
patchtrain = pickle.load(file)
file.close()
file = open('patchtest.pickle','rb')
patchtest = pickle.load(file)
file.close()

In [ ]:
patchtrain[0][0]

In [ ]:
scattering = Scattering2D(J=2, shape=(180,180)) 

In [6]:
alltrain = [arr[0] for arr in patchtrain.values()]
alltest = [arr[0] for arr in patchtest.values()]

In [17]:
fourdtrain = np.concatenate([arr[np.newaxis] for arr in alltrain])
fourdtest = np.concatenate([arr[np.newaxis] for arr in alltest])

In [19]:
np.mean(fourdtrain,axis = (0,1,2))

array([ 93.21681241,  42.66866576,  28.09202537])

In [20]:
np.mean(fourdtest,axis = (0,1,2))

array([ 91.18725809,  42.20415004,  27.21451807])

In [21]:
np.std(fourdtrain,axis = (0,1,2))

array([ 39.38056547,  18.04536693,  11.35980616])

In [22]:
np.std(fourdtest,axis = (0,1,2))

array([ 41.22684355,  18.94664187,  11.76115604])

In [8]:
# Get scattering representation 
p = ep('/scratch/jx1047/project/Scattering-colonography/data/e_optha/'
       ,transform = transforms.Compose([transforms.RandomResizedCrop(40),transforms.RandomHorizontalFlip(),transforms.ToTensor()
                                        ,transforms.Normalize((93.21681241,  42.66866576,  28.09202537),(39.38056547,  18.04536693,  11.35980616))])
       ,train = True,load_prob = True,green = False
       ,rescale = 'intensity')
p2 = ep('/scratch/jx1047/project/Scattering-colonography/data/e_optha/'
       ,transform = transforms.Compose([transforms.RandomResizedCrop(40),transforms.RandomHorizontalFlip(),transforms.ToTensor()
                                        ,transforms.Normalize((91.18725809,  42.20415004,  27.21451807),(41.22684355,  18.94664187,  11.76115604))])
       ,train = False,load_prob = True,green = False
       ,rescale = 'intensity')
use_cuda = torch.cuda.is_available()
device = torch.cuda.device("cuda:0" if use_cuda else "cpu")


params = {'batch_size': 1, 
      'shuffle': True,
      'num_workers': 4}
training_generator = data.DataLoader(p, **params)
testing_generator = data.DataLoader(p2,**params)



In [10]:
scattering = Scattering2D(J=2, shape=(40,40)) 

In [30]:

for i, a in enumerate(training_generator, 0):
    ex = scattering(a[0])
    if i == 0:
        xtrain = ex.numpy().flatten()
        ytrain = a[1].numpy()
    else:
        xtrain = np.vstack((xtrain,ex.numpy().flatten()))
        ytrain = np.vstack((ytrain,a[1].numpy()))
        
    if i %50 == 0:
        print(i)
ytrain = ytrain.reshape(-1)    

/home/jx1047/.conda/envs/image/lib/python3.6/site-packages/skimage/exposure/exposure.py:297: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)
/home/jx1047/.conda/envs/image/lib/python3.6/site-packages/skimage/exposure/exposure.py:297: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)
/home/jx1047/.conda/envs/image/lib/python3.6/site-packages/skimage/exposure/exposure.py:297: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)
/home/jx1047/.conda/envs/image/lib/python3.6/site-packages/skimage/exposure/exposure.py:297: RuntimeWarning: invalid value encountered in true_divide
  image = (image - imin) / float(imax - imin)


In [32]:
ytrain.shape

(2656,)

In [33]:
with open('scatpatchtrainx.pickle','wb') as handle:
    pickle.dump(xtrain,handle)
with open('scatpatchtrainy.pickle','wb') as handle:
    pickle.dump(ytrain,handle)

In [11]:
for i, a in enumerate(testing_generator, 0):
    ex = scattering(a[0])
    if i == 0:
        xtest = ex.numpy().flatten()
        ytest = a[1].numpy()
    else:
        xtest = np.vstack((xtest,ex.numpy().flatten()))
        ytest = np.vstack((ytest,a[1].numpy()))
ytest = ytest.reshape(-1)
with open('scatpatchtestx.pickle','wb') as handle:
    pickle.dump(xtest,handle)
with open('scatpatchtesty.pickle','wb') as handle:
    pickle.dump(ytest,handle)

In [12]:
file = open('scatpatchtrainx.pickle','rb')
xtrain = pickle.load(file)
file.close()
file = open('scatpatchtrainy.pickle','rb')
ytrain = pickle.load(file)
file.close()
file = open('scatpatchtestx.pickle','rb')
xtest = pickle.load(file)
file.close()
file = open('scatpatchtesty.pickle','rb')
ytest = pickle.load(file)
file.close()

In [15]:
scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
scaler = StandardScaler()
scaler.fit(xtrain)
scaler.transform(xtrain)
scaler.transform(xtest)
# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score.
# That estimator is made available at ``gs.best_estimator_`` along with
# parameters like ``gs.best_score_``, ``gs.best_params_`` and
# ``gs.best_index_``
pipe = Pipeline([
    # the reduce_dim stage is populated by the param_grid
    ('reduce_dim', PCA()),
    ('classify', SVC())
])


param_grid = dict(reduce_dim__n_components=[1500,1600,1700,1800],classify__C = [1, 10, 100, 1000],classify__kernel = ['rbf','linear'])
grid = GridSearchCV(pipe, cv=5, n_jobs=1, 
                    param_grid=param_grid,scoring=scoring, refit='AUC', return_train_score=True)

grid.fit(xtrain, ytrain)
results = grid.cv_results_


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
#Test:
testpipe = Pipeline([
    ('reduce_dim', PCA()), #best_parameters
    ('classify', SVC())
])

testpipe.fit(xtrain,ytrain)
pred = testpipe.predict_proba(xtest)
roc_auc_score(y_test, pred)
